In [29]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [30]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [31]:
pip install tf-nightly

Note: you may need to restart the kernel to use updated packages.


In [32]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sanga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
dataset = load_dataset("cnn_dailymail", version ="3.0.0" )

  0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
print(f"features in cnn_dailymail :{dataset['train'].column_names}")

features in cnn_dailymail :['article', 'highlights', 'id']


In [35]:
print(dataset.shape)

{'train': (287113, 3), 'validation': (13368, 3), 'test': (11490, 3)}


In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [37]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(f'\nSummary (length: {len(sample["highlights"])}):')


Article (excerpt of 500 characters, total length: 4051):


Summary (length: 281):


In [38]:
sample_text = dataset["train"][1]["article"][:2000]

# We'll collect the generated summaries of each model in a dictionary
summaries = {}

In [39]:
def baseline_summary_three_sent(text):
    return "\n".join(sent_tokenize(text)[:3])

In [40]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)

In [41]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipeline('text-generation', model = 'gpt2-medium' )

gpt2_query = sample_text + "\nTL;DR:\n"

pipe_out = pipe(gpt2_query, max_length = 512, clean_up_tokenization_spaces = True)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [42]:
pipe_out

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [43]:
pipe_out[0]["generated_text"][len(gpt2_query) :]

"- Prisoners are wearing sleeveless prison clothes with holes cut into the tops. The holes fit on them and they can't walk. People are unable to properly control their emotions and the prison environment can become a nightmare. -\n- Most people in state prisons in Florida are mentally ill. \n- The hole in their top has been cut off at wrist level and now leads"

In [44]:
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))
summaries['gpt2']

"- Prisoners are wearing sleeveless prison clothes with holes cut into the tops.\nThe holes fit on them and they can't walk.\nPeople are unable to properly control their emotions and the prison environment can become a nightmare.\n-\n- Most people in state prisons in Florida are mentally ill. \n- The hole in their top has been cut off at wrist level and now leads"

In [45]:
print("GROUND TRUTH")

print(dataset['train'][1]['highlights'])


for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
GPT2
- Prisoners are wearing sleeveless prison clothes with holes cut into the tops.
The holes fit on them and they can't walk.
People are unable to properly control their emotions and the prison environment can become a nigh

In [46]:
def calculate_metric_on_baseline_test_ds(dataset, metric, column_text = 'article', column_summary = 'highlights' ):
   
    summaries = [baseline_summary_three_sent(text) for text in dataset[column_text] ]

    metric.add_batch(predictions = summaries, references = dataset[column_summary] )

    score = metric.compute()
    return score

In [47]:
test_sampled = dataset['train'].shuffle(seed = 42).select(range(1000))

score = calculate_metric_on_baseline_test_ds(test_sampled, rouge_metric )

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame.from_dict(rouge_dict, orient = 'index' , columns = ['baseline'] ).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.253995,0.100642,0.165754,0.231571


In [48]:
bleu_metric = load_metric("sacrebleu")
bleu_metric.add(prediction = [summaries["gpt2"]], reference = [dataset['train'][1]['highlights'] ])

results = bleu_metric.compute(smooth_method = 'floor', smooth_value = 0 )

results['precision'] = [np.round(p , 2) for p in results['precisions'] ]

pd.DataFrame.from_dict(results, orient = 'index', columns = ['Value'] )

,Value
score,0.0
counts,"[12, 0, 0, 0]"
totals,"[79, 78, 77, 76]"
precisions,"[15.189873417721518, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,79
ref_len,57
precision,"[15.19, 0.0, 0.0, 0.0]"


In [49]:
rouge_metric = load_metric('rouge')
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

reference = dataset['train'][1]['highlights']

records = []

for model_name in summaries:
    rouge_metric.add(prediction = summaries[model_name], reference = reference )
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index = summaries.keys() )

rouge_dict  {'rouge1': 0.365079365079365, 'rouge2': 0.14516129032258066, 'rougeL': 0.20634920634920634, 'rougeLsum': 0.2857142857142857}
rouge_dict  {'rouge1': 0.23214285714285715, 'rouge2': 0.018181818181818184, 'rougeL': 0.125, 'rougeLsum': 0.2142857142857143}


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.232143,0.018182,0.125000,0.214286
